<a href="https://colab.research.google.com/github/Mohamed-derbeli/Machine-Learning-Projects/blob/main/Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Price Prediction**


*   In this project, we will be talking about predicting the returns on stocks. 
*   We will predict the stock price using the LSTM neural network.
*   The data contains records about the stock price of Tata Global Beverages Limited. The dataset also contains a date-wise price of stock with open, close, high, and low prices along with volume traded as well as turnover on that day.

### 1.    Imports

In [ ]:
import os
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
# from keras.models import Sequential
# from keras.layers import LSTM,Dropout,Dense
# from sklearn.preprocessing import MinMaxScaler


### 3. Connection to mysql server

**Step 1: **Install Jupyter
Install Jupyter on your local machine.

**Step 2:** Install and enable the jupyter_http_over_ws jupyter extension (one-time)

The jupyter_http_over_ws extension is authored by the Colaboratory team and available on GitHub.

pip install jupyter_http_over_ws

jupyter serverextension enable --py jupyter_http_over_ws

**Step 3:** Start server and authenticate
New notebook servers are started normally, though you will need to set a flag to explicitly trust WebSocket connections from the Colaboratory frontend.

jupyter notebook \

  --NotebookApp.allow_origin='https://colab.research.google.com' \

  --port=8888 \

  --NotebookApp.port_retries=0
    
Once the server has started, it will print a message with the initial backend URL used for authentication. Make a copy of this URL as you'll need to provide this in the next step.

**Step 4:** Connect to the local runtime
In Colaboratory, click the "Connect" button and select "Connect to local runtime...". Enter the URL from the previous step in the dialog that appears and click the "Connect" button. After this, you should now be connected to your local runtime.

In [ ]:
# Set environment variables
os.environ['MYSQL_DB_USERNAME'] = input("Enter the username of the environment variable:")
os.environ['MYSQL_DB_PASSWORD'] = input("Enter the password of the environment variable:")

# Get environment variables
user_name = os.environ.get('MYSQL_DB_USERNAME')
password = os.environ.get('MYSQL_DB_PASSWORD')

_

In [ ]:
# DEFINE THE DATABASE CREDENTIALS
user = user_name
password = password
host = 'localhost'
port = 3306
database = 'schema_2_'
  
# PYTHON FUNCTION TO CONNECT TO THE MYSQL DATABASE AND
engine= create_engine(url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database))
connection=engine.connect()
  



## 2.   Read the dataset:

In [ ]:
query = """SELECT * FROM schema_2_.Stock_Price_Predicton;"""

sql_df = pd.read_sql(query, con=connection)

In [ ]:
sql_df.head()